In [1]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
import ast
import numpy as np
import altair as alt
import re
import pickle as pkl
import matplotlib


In [2]:
with open("../results/adressa_results_k@10.pkl", "rb") as f:
    results = pkl.load(f)

results_mean = results.map(
    lambda x: np.mean(x) if isinstance(x, (list, np.ndarray)) and len(x) > 0 else np.nan
)

display(results_mean)

/var/folders/f6/ggzrtr6d13l0g85_pyz_1zhm0000gp/T/ipykernel_68754/4023409000.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  results_mean = results.applymap(


,topic_calibration,subtopic_calibration,complexity_calibration,fragmentation,activation,representation,alternative_voices,tf_idf_ild,sentbert_ild,gini,ndcg_values
pop,0.478471,0.692206,0.418422,0.276361,0.219273,NaN,NaN,0.993032,0.602110,0.283383,0.249076
random,0.525552,0.705913,0.420120,0.230024,0.218839,NaN,NaN,0.994001,0.608864,0.290931,0.222570
nrms,0.525614,0.707023,0.416924,0.129986,0.218425,NaN,NaN,0.994083,0.610385,0.289221,0.227493
lstur,0.525472,0.706445,0.418634,0.152764,0.217508,NaN,NaN,0.994011,0.610379,0.289703,0.232621
naml,0.526615,0.707392,0.419117,0.134058,0.217981,NaN,NaN,0.994073,0.609984,0.290953,0.227439


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
legend_name_map = {
    'pop': 'Popularity',
    'random': 'Random',
    'npa': 'NPA',
    'nrms': 'NRMS',
    'lstur': 'LSTUR',
    'naml': 'NAML',
}
ordered_recommenders = ['lstur', 'naml', 'nrms', 'npa', 'pop', 'random']
metric_specs = [
    {'label': 'Topic Calibration', 'x_label': 'Topic Calibration', 'path': Path('../results/adressa_topic_tradeoff_k@10.csv')},
    {'label': 'Subtopic Calibration', 'x_label': 'Subtopic Calibration', 'path': Path('../results/adressa_subtopic_tradeoff_k@10.csv')},
    {'label': 'Complexity Calibration', 'x_label': 'Complexity Calibration', 'path': Path('../results/adressa_complexity_tradeoff_k@10.csv')},
    {'label': 'Fragmentation', 'x_label': 'Fragmentation', 'path': Path('../results/adressa_fragmentation_tradeoff_k@10.csv')},
    {'label': 'Activation', 'x_label': 'Activation', 'path': Path('../results/adressa_activation_tradeoff_k@10.csv')},
]
palette = matplotlib.colormaps['tab10']
color_map = {rec: palette(i) for i, rec in enumerate(ordered_recommenders)}
fig, axes = plt.subplots(2, 3, figsize=(18, 12), sharey=True)
axes = axes.flatten()
legend_handles = {}
for ax, spec in zip(axes, metric_specs):
    tradeoff_df = pd.read_csv(spec['path'])
    tradeoff_df = tradeoff_df.dropna(subset=['ndcg', 'divergence'])
    tradeoff_df = tradeoff_df.sort_values(['recommender', 'lambda'])
    plot_df = tradeoff_df[tradeoff_df['recommender'] != 'incorrect_random']
    for rec in ordered_recommenders:
        group = plot_df[plot_df['recommender'] == rec]
        if group.empty:
            continue
        display_name = legend_name_map.get(rec, rec)
        # Add a tiny jitter when multiple lambdas land on the exact same point (shift right only)
        jittered = group.copy()
        seen_coords = {}
        jitter_step = 2.5e-3
        jitter_margin = 1e-2
        for idx, row in jittered.iterrows():
            key = (
                round(row['divergence'] / jitter_margin),
                round(row['ndcg'] / jitter_margin),
            )
            offset = seen_coords.get(key, 0)
            if offset:
                jittered.at[idx, 'divergence'] = row['divergence'] + jitter_step * offset
            seen_coords[key] = offset + 1
        line, = ax.plot(
            jittered['divergence'],
            jittered['ndcg'],
            linewidth=3.5,
            marker='o',
            markersize=8,
            color=color_map[rec],
            label=display_name,
        )
        legend_handles[display_name] = line
        # Label each point with its lambda value
        for x, y, lam in zip(jittered['divergence'], jittered['ndcg'], jittered['lambda']):
            ax.annotate(
                f"{lam}",
                (x, y),
                textcoords="offset points",
                xytext=(5, 4),
                ha="left",
                va="center",
                fontsize=9,
                color=color_map[rec],
                alpha=0.8,
            )
    ax.set_xlabel(spec['x_label'], fontsize=18)
    ax.set_ylabel('NDCG@10', fontsize=18)
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.grid(True, linestyle='--', alpha=0.5)
for ax in axes[len(metric_specs):]:
    ax.axis('off')
fig.legend(
    legend_handles.values(),
    legend_handles.keys(),
    loc='lower center',
    ncol=3,
    fontsize=16,
    title='Recommender',
    title_fontsize=16,
    bbox_to_anchor=(0.5, -0.04),
)
plt.tight_layout(rect=[0, 0.08, 1, 1])
plt.subplots_adjust(bottom=0.12)
plt.show()
